In [2]:
# Import Dependencies
import cv2
import mediapipe as mp
import time
import vlc
import numpy as np
import time

In [3]:
def Pause_Instance(Status):
    Raised = {
    "Left" : {
        "INDEX"  : True,
        "MIDDLE" : True,
        "RING"   : True,
        "PINKY"  : True,
        "THUMB"  : True
    }
    }
    if Raised["Left"] == Status["Left"]:
        return True
    return False

In [4]:
def Vol_Instance(Status):
    Raised = {
    "Right" : {
        "INDEX"  : True,
        "MIDDLE" : False,
        "RING"   : False,
        "PINKY"  : False,
        "THUMB"  : False
    }
    }
    if Raised["Right"] == Status["Right"]:
        return True
    return False

In [7]:

###################################################  SETUP OPENCV AND MEDIAPIPE ################################################
cap = cv2.VideoCapture(0) # 0 --> Webcam

# Create Instance of Mediapipe Hands Class
mpHands = mp.solutions.hands

# Initialize Drawing Class
mpDraw = mp.solutions.drawing_utils

# For Videos
Hands = mpHands.Hands(static_image_mode = False,max_num_hands = 1,min_detection_confidence = 0.65, min_tracking_confidence = 0.5)

################################################################################################################################

###################################################### SETUP VLC INSTANCE  #####################################################
vlc_instance = vlc.Instance()

# creating a media player
player = vlc_instance.media_player_new()

# creating a media
media = vlc_instance.media_new(r"E:\Movies\Venom.mp4")

# setting media to the player
player.set_media(media)
player.audio_set_volume(70)

# play the video
player.play()
################################################################################################################################

###################################################  STANCE CHECKING FLAGS   ###################################################
Init = 0
pause_check = 0
vol_flag = 0
pause = 0
Status = {
    "Left" : {
        "INDEX"  : False,
        "MIDDLE" : False,
        "RING"   : False,
        "PINKY"  : False,
        "THUMB"  : False
    },
            "Right" : {
        "INDEX"  : False,
        "MIDDLE" : False,
        "RING"   : False,
        "PINKY"  : False,
        "THUMB"  : False
    }
}

################################################################################################################################

while cap.isOpened():
    flag, img = cap.read()
    img = cv2.flip(img,1) # Horizontal Flip for Mirror Effect

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Mediapipe model only works with RGB image
    
    detectedHand = Hands.process(imgRGB)
    
    FingerTips_ids = [mpHands.HandLandmark.PINKY_TIP,
                      mpHands.HandLandmark.MIDDLE_FINGER_TIP,
                      mpHands.HandLandmark.RING_FINGER_TIP,
                      mpHands.HandLandmark.INDEX_FINGER_TIP]
    
    if detectedHand.multi_hand_landmarks:
        for hand_index, hand_info in enumerate(detectedHand.multi_handedness):

            # Detect if Hand is LEFT or RIGHT
            hand_label = hand_info.classification[0].label

            # Loacate Detected Handlandmarks
            hand_ldmrks = detectedHand.multi_hand_landmarks[hand_index]

            # Check Fingers Status
            if not Init:
                Init = 15 # Detect Status once in 15 frames
                for tip in FingerTips_ids:
                    if (hand_ldmrks.landmark[tip].y < hand_ldmrks.landmark[tip-2].y):
                        Status[str(hand_label)][str(tip.name.split("_")[0])] = True
#                 print(Status)
                thumb_ptr = mpHands.HandLandmark.THUMB_TIP
                Status[str(hand_label)]["THUMB"] = True if hand_ldmrks.landmark[thumb_ptr].x >= hand_ldmrks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP-2].x else False
            else:
                Init -= 1
            
            if not pause_check and Pause_Instance(Status):
                # Pause and Play 
                player.pause()
                pause = 1 if pause == 0 else 0
                
                pause_check = 5
                
                
            
            if Vol_Instance(Status):
                print("Here",vol_flag)
                if not vol_flag:
                    OG_A = hand_ldmrks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
                    OG_B = hand_ldmrks.landmark[mpHands.HandLandmark.THUMB_TIP]
                    OG_dis = np.sqrt(((OG_A.x-OG_B.x))**2 + ((OG_A.y-OG_B.y))**2)
                    vol_flag = 15
                else:
                    A = hand_ldmrks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
                    B = hand_ldmrks.landmark[mpHands.HandLandmark.THUMB_TIP]
                    dis = np.sqrt(((A.x-B.x))**2 + ((A.y-B.y))**2)
                    print("{:.2f} {:.2f}".format(OG_dis,dis), f"Volume = {np.clip(player.audio_get_volume() + int((OG_dis-dis)*-15),0,200):.2f}")
                    player.audio_set_volume(np.clip(player.audio_get_volume() + int((OG_dis-dis)*-15),0,200))
                    cv2.putText(img, "VOL: "+str(player.audio_get_volume()), (500,450),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,160,255), 2, cv2.LINE_AA)
                    cv2.rectangle(img, (10,100), (50,450), (220,20,60),1)
                    cv2.rectangle(img,(10,450-int(player.audio_get_volume()*350/200)),(50,450),(220,20,60),-1)

            
            # Reset Instance checkers
            if not Pause_Instance(Status) and pause_check > 0:
#                 print("Here")
                pause_check -= 1
#                 print("Play")
            if not Vol_Instance(Status) and vol_flag :
                vol_flag -= 1
#                  
            
#     Data Rendering
            
            mpDraw.draw_landmarks((img),hand_ldmrks, mpHands.HAND_CONNECTIONS) # Draw on the Image we are Displaying
            
    else:
        Status = {
    "Left" : {
        "INDEX"  : False,
        "MIDDLE" : False,
        "RING"   : False,
        "PINKY"  : False,
        "THUMB"  : False
    },
            "Right" : {
        "INDEX"  : False,
        "MIDDLE" : False,
        "RING"   : False,
        "PINKY"  : False,
        "THUMB"  : False
    }
    }
    # View our model's working
    cv2.putText(img, "STATUS :  " + str(["Play","Pause"][pause]), (20,70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (60,20,220), 2, cv2.LINE_AA)
    cv2.imshow("Image",img)
    
    # Termination Statement
    if cv2.waitKey(2) & 0xFF == ord('e'):
        player.release()
        break
        
        
# Destroy any Unnecessary Windows opened by cv2            
cv2.destroyAllWindows()
# Release Webcam
cap.release()

Here 0
Here 15
0.12 0.12 Volume = 70.00
Here 15
0.12 0.12 Volume = 70.00
Here 15
0.12 0.13 Volume = 70.00
Here 15
0.12 0.14 Volume = 70.00
Here 15
0.12 0.15 Volume = 70.00
Here 15
0.12 0.15 Volume = 70.00
Here 15
0.12 0.16 Volume = 70.00
Here 15
0.12 0.17 Volume = 70.00
Here 15
0.12 0.18 Volume = 70.00
Here 15
0.12 0.18 Volume = 70.00
Here 15
0.12 0.20 Volume = 71.00
Here 15
0.12 0.19 Volume = 72.00
Here 15
0.12 0.20 Volume = 73.00
Here 15
0.12 0.20 Volume = 74.00
Here 15
0.12 0.21 Volume = 75.00
Here 15
0.12 0.21 Volume = 76.00
Here 15
0.12 0.23 Volume = 77.00
Here 15
0.12 0.23 Volume = 78.00
Here 15
0.12 0.23 Volume = 79.00
Here 15
0.12 0.23 Volume = 80.00
Here 15
0.12 0.24 Volume = 81.00
Here 15
0.12 0.24 Volume = 82.00
Here 15
0.12 0.24 Volume = 83.00
Here 15
0.12 0.24 Volume = 84.00
Here 15
0.12 0.25 Volume = 85.00
Here 15
0.12 0.25 Volume = 86.00
Here 15
0.12 0.25 Volume = 87.00
Here 15
0.12 0.25 Volume = 88.00
Here 15
0.12 0.25 Volume = 89.00
Here 15
0.12 0.26 Volume = 91.00
Her

Here 15
0.07 0.07 Volume = 200.00
Here 15
0.07 0.08 Volume = 200.00
Here 15
0.07 0.08 Volume = 200.00
Here 15
0.07 0.09 Volume = 200.00
Here 15
0.07 0.08 Volume = 200.00
Here 15
0.07 0.10 Volume = 200.00
Here 15
0.07 0.13 Volume = 200.00
Here 15
0.07 0.12 Volume = 200.00
Here 15
0.07 0.14 Volume = 200.00
Here 15
0.07 0.14 Volume = 200.00
Here 15
0.07 0.15 Volume = 200.00
Here 15
0.07 0.15 Volume = 200.00
Here 15
0.07 0.16 Volume = 200.00
Here 15
0.07 0.16 Volume = 200.00
Here 15
0.07 0.18 Volume = 200.00
Here 15
0.07 0.18 Volume = 200.00
Here 15
0.07 0.18 Volume = 200.00
Here 15
0.07 0.20 Volume = 200.00
Here 15
0.07 0.20 Volume = 200.00
Here 15
0.07 0.21 Volume = 200.00
Here 15
0.07 0.21 Volume = 200.00
Here 15
0.07 0.21 Volume = 200.00
Here 15
0.07 0.21 Volume = 200.00
Here 15
0.07 0.22 Volume = 200.00
Here 15
0.07 0.23 Volume = 200.00
Here 15
0.07 0.22 Volume = 200.00
Here 15
0.07 0.23 Volume = 200.00
Here 15
0.07 0.20 Volume = 200.00
Here 15
0.07 0.23 Volume = 200.00
Here 15
0.07 0

Here 5
0.04 0.02 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.04 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.02 Volume = 200.00
Here 5
0.04 0.02 Volume = 200.00
Here 5
0.04 0.02 Volume = 200.00
Here 5
0.04 0.02 Volume = 200.00
Here 5
0.04 0.02 Volume = 200.00
Here 5
0.04 0.02 Volume = 200.00
Here 5
0.04 0.02 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.01 Volume = 200.00
Here 5
0.04 0.01 Volume = 200.00
Here 5
0.04 0.01 Volume = 200.00
Here 5
0.04 0.02 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.02 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.04 0.03 Volume = 200.00
Here 5
0.0

Here 15
0.31 0.29 Volume = 0.00
Here 15
0.31 0.28 Volume = 0.00
Here 15
0.31 0.29 Volume = 0.00
Here 15
0.31 0.29 Volume = 0.00
Here 15
0.31 0.29 Volume = 0.00
Here 15
0.31 0.29 Volume = 0.00
Here 15
0.31 0.29 Volume = 0.00
Here 15
0.31 0.29 Volume = 0.00
Here 15
0.31 0.29 Volume = 0.00
Here 15
0.31 0.29 Volume = 0.00
Here 15
0.31 0.29 Volume = 0.00
Here 15
0.31 0.21 Volume = 0.00
Here 15
0.31 0.20 Volume = 0.00
Here 15
0.31 0.12 Volume = 0.00
Here 15
0.31 0.25 Volume = 0.00
Here 15
0.31 0.20 Volume = 0.00
Here 11
0.31 0.09 Volume = 0.00
Here 11
0.31 0.12 Volume = 0.00
Here 11
0.31 0.11 Volume = 0.00
Here 11
0.31 0.09 Volume = 0.00
Here 11
0.31 0.10 Volume = 0.00
Here 11
0.31 0.11 Volume = 0.00
Here 11
0.31 0.11 Volume = 0.00
Here 11
0.31 0.13 Volume = 0.00
Here 11
0.31 0.12 Volume = 0.00
Here 11
0.31 0.13 Volume = 0.00
Here 11
0.31 0.13 Volume = 0.00
Here 11
0.31 0.15 Volume = 0.00
Here 11
0.31 0.15 Volume = 0.00
Here 11
0.31 0.15 Volume = 0.00
Here 11
0.31 0.16 Volume = 0.00
Here 11


Here 15
0.07 0.07 Volume = 123.00
Here 15
0.07 0.07 Volume = 123.00
Here 15
0.07 0.07 Volume = 123.00
Here 15
0.07 0.07 Volume = 123.00
Here 15
0.07 0.05 Volume = 123.00
Here 15
0.07 0.05 Volume = 123.00
Here 15
0.07 0.05 Volume = 123.00
Here 15
0.07 0.06 Volume = 123.00
Here 15
0.07 0.05 Volume = 123.00
Here 15
0.07 0.05 Volume = 123.00
Here 15
0.07 0.05 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0.03 Volume = 123.00
Here 15
0.07 0.03 Volume = 123.00
Here 15
0.07 0.03 Volume = 123.00
Here 15
0.07 0.03 Volume = 123.00
Here 15
0.07 0.03 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0.03 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0.04 Volume = 123.00
Here 15
0.07 0

Here 15
0.27 0.33 Volume = 44.00
Here 15
0.27 0.35 Volume = 45.00
Here 15
0.27 0.34 Volume = 46.00
Here 15
0.27 0.35 Volume = 47.00
Here 15
0.27 0.35 Volume = 48.00
Here 15
0.27 0.36 Volume = 49.00
Here 15
0.27 0.36 Volume = 50.00
Here 15
0.27 0.36 Volume = 51.00
Here 15
0.27 0.37 Volume = 52.00
Here 15
0.27 0.37 Volume = 53.00
Here 15
0.27 0.37 Volume = 54.00
Here 15
0.27 0.38 Volume = 55.00
Here 15
0.27 0.38 Volume = 56.00
Here 15
0.27 0.38 Volume = 57.00
Here 15
0.27 0.38 Volume = 58.00
Here 15
0.27 0.38 Volume = 59.00
Here 15
0.27 0.38 Volume = 60.00
Here 15
0.27 0.38 Volume = 61.00
Here 15
0.27 0.38 Volume = 62.00
Here 15
0.27 0.38 Volume = 63.00
Here 15
0.27 0.38 Volume = 64.00
Here 15
0.27 0.38 Volume = 65.00
Here 15
0.27 0.39 Volume = 66.00
Here 15
0.27 0.38 Volume = 67.00
Here 15
0.27 0.38 Volume = 68.00
Here 15
0.27 0.39 Volume = 69.00
Here 15
0.27 0.39 Volume = 70.00
Here 15
0.27 0.39 Volume = 71.00
Here 15
0.27 0.39 Volume = 72.00
Here 15
0.27 0.39 Volume = 73.00
Here 15
0.

Here 15
0.27 0.35 Volume = 78.00
Here 15
0.27 0.35 Volume = 79.00
Here 15
0.27 0.35 Volume = 80.00
Here 15
0.27 0.35 Volume = 81.00
Here 15
0.27 0.35 Volume = 82.00
Here 15
0.27 0.35 Volume = 83.00
Here 15
0.27 0.35 Volume = 84.00
Here 15
0.27 0.35 Volume = 85.00
Here 15
0.27 0.35 Volume = 86.00
Here 15
0.27 0.35 Volume = 87.00
Here 15
0.27 0.35 Volume = 88.00
Here 15
0.27 0.35 Volume = 89.00
Here 15
0.27 0.35 Volume = 90.00
Here 15
0.27 0.35 Volume = 91.00
Here 15
0.27 0.35 Volume = 92.00
Here 15
0.27 0.35 Volume = 93.00
Here 15
0.27 0.34 Volume = 94.00
Here 15
0.27 0.34 Volume = 95.00
Here 15
0.27 0.34 Volume = 96.00
Here 15
0.27 0.34 Volume = 97.00
Here 15
0.27 0.34 Volume = 98.00
Here 15
0.27 0.34 Volume = 98.00
Here 15
0.27 0.34 Volume = 99.00
Here 15
0.27 0.33 Volume = 99.00
Here 15
0.27 0.33 Volume = 99.00
Here 15
0.27 0.33 Volume = 99.00
Here 15
0.27 0.33 Volume = 99.00
Here 15
0.27 0.33 Volume = 99.00
Here 15
0.27 0.33 Volume = 99.00
Here 15
0.27 0.33 Volume = 99.00
Here 15
0.

Here 15
0.10 0.16 Volume = 157.00
Here 15
0.10 0.16 Volume = 157.00
Here 15
0.10 0.17 Volume = 157.00
Here 15
0.10 0.17 Volume = 158.00
Here 15
0.10 0.17 Volume = 159.00
Here 15
0.10 0.17 Volume = 160.00
Here 15
0.10 0.18 Volume = 161.00
Here 15
0.10 0.18 Volume = 162.00
Here 15
0.10 0.18 Volume = 163.00
Here 15
0.10 0.20 Volume = 164.00
Here 15
0.10 0.19 Volume = 165.00
Here 15
0.10 0.19 Volume = 166.00
Here 15
0.10 0.20 Volume = 167.00
Here 15
0.10 0.20 Volume = 168.00
Here 15
0.10 0.19 Volume = 169.00
Here 15
0.10 0.19 Volume = 170.00
Here 15
0.10 0.19 Volume = 171.00
Here 15
0.10 0.19 Volume = 172.00
Here 15
0.10 0.12 Volume = 172.00
Here 15
0.10 0.14 Volume = 172.00
Here 15
0.10 0.08 Volume = 172.00
Here 15
0.10 0.07 Volume = 172.00
Here 15
0.10 0.07 Volume = 172.00
Here 15
0.10 0.07 Volume = 172.00
Here 15
0.10 0.07 Volume = 172.00
Here 15
0.10 0.07 Volume = 172.00
Here 15
0.10 0.07 Volume = 172.00
Here 15
0.10 0.07 Volume = 172.00
Here 15
0.10 0.07 Volume = 172.00
Here 15
0.10 0

Here 15
0.10 0.27 Volume = 200.00
Here 15
0.10 0.28 Volume = 200.00
Here 15
0.10 0.26 Volume = 200.00
Here 15
0.10 0.27 Volume = 200.00
Here 15
0.10 0.05 Volume = 200.00
Here 15
0.10 0.14 Volume = 200.00
Here 15
0.10 0.16 Volume = 200.00


In [8]:
cv2.destroyAllWindows()
# Release Webcam
cap.release()

In [9]:
player.release()

# 